In [1]:
!pip install praw
!pip install transformers

In [2]:
import praw

from transformers import AutoTokenizer, BartTokenizer, BartForConditionalGeneration, T5Tokenizer, T5ForConditionalGeneration

/Users/osama/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
ids = {
      "client_id": "tzDk97ujk4gMDdATeYUxYg",
      "client_secret": "S8ZLg2dzg4dWxODm6d_sQAKQoRPuBA",
      "user_agent": "aita_scraper_1"
    }

client_id = ids["client_id"]
client_secret = ids["client_secret"]
user_agent = ids["user_agent"]

In [4]:
def clean_post(text):
    text = text.replace('\n', ' ').lower()
    edit_idx = text.find('edit:')
    if edit_idx != -1:
        text = text[:edit_idx]
    update_idx = text.find('update:')
    if update_idx != -1:
        text = text[:update_idx]
    return text.strip()

In [5]:
def runTest(post_url,model_path, model_type = "bart", withTitle = False, sequence_length = 1024 ):

    reddit = praw.Reddit(client_id=client_id, client_secret=client_secret, user_agent=user_agent)
    
    try:
        post = reddit.submission(url=post_url)
        post_title = clean_post(post.title)
        post_body = clean_post(post.selftext)

        print("Post Title:", post_title)
        print()
        print("Post Body:", post_body)
        print()

        AVAILABLE_MODELS = {
        "bart": "facebook/bart-base", 
        "t5": "google/flan-t5-base"
        }

        model_link = AVAILABLE_MODELS[model_type]

        print("Using settings")
        print("model:", model_type)
        print("with title:", withTitle)
        print("sequence length:", sequence_length)
        print()

        print("Loading model...", model_path)
        print()

        if model_type == "bart":
            model = BartForConditionalGeneration.from_pretrained(model_path)
        else:
            model = T5ForConditionalGeneration.from_pretrained(model_path)

        tokenizer = AutoTokenizer.from_pretrained(model_link)
        
        data = post_body

        if withTitle:
            data = post_title + " " + post_body


        inputs = tokenizer([data], max_length=sequence_length, return_tensors="pt",  truncation=True)

        comments = {}

        beams = [2,10]

        print("Generating comments...")
        print()

        if model_type == "bart":
            for length in [0, 100]:
                name = "greed search comment, comment length - {}".format(length)
                summary_ids = model.generate(inputs["input_ids"], min_length=length, max_length=sequence_length)
                comments[name] = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
                print(name + ":", comments[name])
                print()
                for x in beams:
                    name = "search with {} beams comment, comment length - {}".format(x,length) 
                    summary_ids = model.generate(inputs["input_ids"], num_beams=x, min_length=length, max_length=sequence_length)
                    comments[name] = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
                    print(name + ":", comments[name])
                    print()
        else:
            
            for length in [0, 100]:
                name = "greed search comment, comment length - {}".format(length)
                summary_ids = model.generate(inputs["input_ids"], min_length=length, max_length=sequence_length, repetition_penalty=1.5, no_repeat_ngram_size=3)
                comments[name] = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
                print(name + ":", comments[name])
                print()
                for x in beams:
                    name = "search with {} beams comment, comment length - {}".format(x,length) 
                    summary_ids = model.generate(inputs["input_ids"], num_beams=x, min_length=length, max_length=sequence_length, repetition_penalty=1.5, no_repeat_ngram_size=3)
                    comments[name] = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
                    print(name + ":", comments[name])
                    print()

        # for key, value in comments.items():
        #     print(key + ":", value)
        #     print()

    except Exception as e:
        print("Error:", e)


In [6]:
# Best Bart model
post_url = "https://www.reddit.com/r/AmItheAsshole/comments/12szhlr/aita_for_pressuring_my_so_to_change_her_dress/"
model_path = "/Users/osama/Downloads/finalModels/bart/bart_balanced_subset_without_title_less_than_1024_double_comment"

runTest(post_url, model_path)

Version 7.6.1 of praw is outdated. Version 7.7.0 was released Saturday February 25, 2023.


Post Title: aita for pressuring my so to change her dress before a professional event?

Post Body: this morning my so was dressing up for a research symposium she's attending today. the one in which her senior class presented their research took place on monday, but this time around it is the underclassmen who are presenting their research. on monday, the seniors were instructed to dress in business casual, but today there was no explicit dress code for those just attending the event.      given this is still a professional academic event, i expected the norm to still be business casual. my so went with a dress that looked very much [like this](https://i.imgur.com/qtlvgd5.png) but a much brighter yellow, and sleeves that only went to mid-forearm.  i told her that maybe she shouldn't wear this dress and go with something business casual instead, to which she insisted that this was business casual. i replied saying it was casual but not business, and we went back and forth like that a li

In [7]:
# Best T5 model
post_url = "https://www.reddit.com/r/AmItheAsshole/comments/12szhlr/aita_for_pressuring_my_so_to_change_her_dress/"
model_path = "/Users/osama/Downloads/finalModels/t5/t5_balanced_subset_with_title_less_than_512_double_comment"

runTest(post_url, model_path, model_type = "t5", withTitle = True, sequence_length = 512)

Post Title: aita for pressuring my so to change her dress before a professional event?

Post Body: this morning my so was dressing up for a research symposium she's attending today. the one in which her senior class presented their research took place on monday, but this time around it is the underclassmen who are presenting their research. on monday, the seniors were instructed to dress in business casual, but today there was no explicit dress code for those just attending the event.      given this is still a professional academic event, i expected the norm to still be business casual. my so went with a dress that looked very much [like this](https://i.imgur.com/qtlvgd5.png) but a much brighter yellow, and sleeves that only went to mid-forearm.  i told her that maybe she shouldn't wear this dress and go with something business casual instead, to which she insisted that this was business casual. i replied saying it was casual but not business, and we went back and forth like that a li